# Text preprocessing

During text preprocessing, a corpus of documents is tokenized (i.e. the document strings are split into individual words, punctuation, numbers, etc.) and then these tokens can be transformed, filtered or annotated. The goal is to prepare the raw texts in a way that makes it easier to perform eventual analysis methods in a later stage, e.g. by reducing noise in the dataset. tmtoolkit provides a rich set of tools for this purpose in the [tmtoolkit.preprocess](api.rst#tmtoolkit-preprocess) module.   

## Parallel processing with the TMPreproc class

You can pass a dict-like dataset (i.e. anything that maps document labels to their plain text contents, e.g. a tmtoolkit [Corpus](text_corpora.ipynb) object) to the [TMPreproc class](api.rst#tmpreproc-class-for-parallel-text-preprocessing) and can then then apply several text processing methods to it. You can chain these processing steps by applying one method after another and examine the results.

Under the hood, the [spaCy](https://spacy.io/) package is used to perform NLP methods. However, `TMPreproc` offers much more functionality than spaCy, including flexible token and document filtering. The most important advantage of using `TMPreproc` is that it employs parallel processing, i.e. it uses all available processors on your machine to do the computations necessary during preprocessing. For large text corpora, this can lead to a strong speed up.

<div class="alert alert-info">

Using the functional API
    
Apart from the `TMPreproc` class, tmtoolkit also provides several functions in the [tmtoolkit.preprocess](api.rst#tmtoolkit-preprocess) module. Most of these functions accept a list of spaCy documents along with additional parameters. You may use these functions for quick prototyping, but it is generally much more convenient to use `TMPreproc`. Note that only the latter provides parallel processing.

</div>

### Loading example data

Let's load a sample of three documents from the built-in *NewsArticles* dataset. We'll use only a small number of documents here to have a better overview at the beginning. We can later use a larger sample.

In [ ]:
import random
random.seed(20191018)   # to make the sampling reproducible

from tmtoolkit.corpus import Corpus
from tmtoolkit.preprocess import tokenize

corpus_small = Corpus.from_builtin_corpus('en-NewsArticles').sample(3)

### Optional: enabling logging output

By default, tmtoolkit does not expose any internal logging messages. Sometimes, for example for diagnostic output during debugging or in order to see progress for long running operations, it's helpful to enable logging output display which can be done as follows:

    import logging
    
    logging.basicConfig(level=logging.INFO)
    tmtoolkit_log = logging.getLogger('tmtoolkit')
    # set the minimum log level to display, for instance also logging.DEBUG
    tmtoolkit_log.setLevel(logging.INFO)
    tmtoolkit_log.propagate = True

### Creating a `TMPreproc` object

You can create a `TMPreproc` object (also known as "instance") by passing a dict that maps document labels to (untokenized) documents. Since a tmtoolkit [Corpus](text_corpora.ipynb) behaves like a dict, we can pass our `corpus_small` object. We also need to specify the corpus language as two-letter ISO 639-1 language code (here `"en"` for English).

In [ ]:
from tmtoolkit.preprocess import TMPreproc

preproc = TMPreproc(corpus_small, language='en')
preproc

The above will at first distribute all documents to several sub-processes which will later be used to run the computations in parallel. The number of sub-processes can be controlled via `n_max_processes`. It defaults to the number of CPU cores in your machine. The distribution of documents to the processes happens according to the document size. E.g. when you have two CPU cores, one very large document and three small documents, CPU 1 will take care about the large document alone and CPU 2 will take the other three small documents. After distribution of the documents, they will directly be tokenized (in parallel). Hence when you have a large corpus, the creation of a `TMPreproc` object may take some time because of the tokenization process.

Our `TMPreproc` object `preproc` is now set up to work with the documents passed in `corpus_small` and the language `'en'` for English. All further operations with this object will use the specified documents and language. All documents are directly tokenized.

The method `print_summary()` is very handy and we will use it quite often. It displays a small summary of the documents in the `TMPreproc` object:

In [ ]:
preproc.print_summary()

### Accessing tokens, vocabulary and other important properties

`TMPreproc` provides several *properties* to access its data and some summary statistics. First, let's have a look at the labels (names) of the documents:

In [ ]:
preproc.doc_labels

We can access the tokens of each document by using the [tokens](api.rst#tmtoolkit.preprocess.TMPreproc.tokens) property:

In [ ]:
# use [:10] slice to show only the first 10 tokens
preproc.tokens['NewsArticles-1880'][:10]

If you prefer a tabular output, you can also access the tokens and their metadata as [pandas DataFrames](https://pandas.pydata.org/) or [datatable Frames](https://github.com/h2oai/datatable/).

<div class="alert alert-info">

A note on the use of datatable Frames

If you have installed the [datatable package](https://pypi.org/project/datatable/), many functions and methods in tmtoolkit return or accept [datatable Frames](https://github.com/h2oai/datatable/) instead of (the more commonly known) [pandas DataFrames](https://pandas.pydata.org/). This is because the former is much faster and memory efficient in most cases. You can always convert between the both like this:

    import datatable as dt
    import pandas as pd
    
    # a pandas DataFrame:
    df = pd.DataFrame({'a': [1, 2, 3], 'b': list('xyz')})
    
    # DataFrame to datatable:
    dtable = dt.Frame(df)
    
    # and vice versa datatable to DataFrame:
    df == dtable.to_pandas()
    
    # Out:
    #       a     b
    # 0  True  True
    # 1  True  True
    # 2  True  True

Even first creating a datatable and then converting to a DataFrame is often faster than directly creating a DataFrame.
    
</div>

You can use the [tokens_dataframe](api.rst#tmtoolkit.preprocess.TMPreproc.tokens_dataframe) or [tokens_datatable](api.rst#tmtoolkit.preprocess.TMPreproc.tokens_datatable) properties for tabular output. The datatable Frame consists of at least five columns: The document label, the position of the token in the document (zero-indexed) and token itself, `lemma` and `whitespace`. The `lemma` column contains the token's lemma and `whitespace` indicates whether there is a whitespace *after* the token in the text. Please note that for large amounts of data, `tokens_datatable` is usually quicker than using `tokens_dataframe`.

In [ ]:
preproc.tokens_datatable

More columns may be shown when you add token metadata (more on that later).

The *method* [get_tokens()](api.rst#tmtoolkit.preprocess.TMPreproc.get_tokens) gives you more options for accessing the tokens. For example, you can get all tokens with their metadata as nested dictionary in the form *document label -> metadata key (e.g. "lemma") -> metadata*.

In [ ]:
doctokens = preproc.get_tokens(with_metadata=True, as_datatables=False)
doctokens['NewsArticles-1880'].keys()

In [ ]:
# lemmata for the first 10 tokens in this document
doctokens['NewsArticles-1880']['lemma'][:10]

The following gives you the number of documents and number of unique tokens respectively:

In [ ]:
preproc.n_docs

In [ ]:
preproc.n_tokens

We can also access the number of tokens in each document via [doc_lengths](api.rst#tmtoolkit.preprocess.TMPreproc.doc_lengths) property:

In [ ]:
# displaying only a single document's length here
preproc.doc_lengths['NewsArticles-1880']

The vocabulary is the set of unique tokens in the corpus, i.e. all tokens that occur at least once in at least one of the documents. You can use [.vocabulary](api.rst#tmtoolkit.preprocess.TMPreproc.vocabulary) for that and [.vocabulary_counts](api.rst#tmtoolkit.preprocess.TMPreproc.vocabulary) to additionally get the number of times each token appears in the corpus. 

In [ ]:
preproc.vocabulary[:10]  # displaying only the first 10 here

In [ ]:
# number of unique tokens in all documents
preproc.vocabulary_size

In [ ]:
# how often the word "the" occurs in the whole corpus
preproc.vocabulary_counts['the']

The latter returns a Python [Counter](https://docs.python.org/3/library/collections.html#collections.Counter) object so we can apply its useful functions, e.g. in order to get the most often used tokens:

In [ ]:
preproc.vocabulary_counts.most_common()[:10]

The document frequency of a token is the number of documents in which this token occurs at least once. The properties [vocabulary_abs_doc_frequency](api.rst#tmtoolkit.preprocess.TMPreproc.vocabulary_abs_doc_frequency) and [vocabulary_rel_doc_frequency](api.rst#tmtoolkit.preprocess.TMPreproc.vocabulary_rel_doc_frequency) return this measure as absolute frequency or proportion respectively:

In [ ]:
(preproc.vocabulary_abs_doc_frequency['Trump'],
 preproc.vocabulary_rel_doc_frequency['Trump'])

In [ ]:
(preproc.vocabulary_abs_doc_frequency['Russia'],
 preproc.vocabulary_rel_doc_frequency['Russia'])

### Part-of-Speech (POS) tagging

Part-of-speech (POS) tagging finds the grammatical word-category for each token in a document. The method [pos_tag()](api.html#tmtoolkit.preprocess.TMPreproc.pos_tag) employs this for the whole corpus. The found POS tags are added as metadata to each token. These tags conform to a specific *tagset* which is explained in the [spaCy documentation](https://spacy.io/api/annotation#pos-tagging).

These tags can be used to annotate and filter the documents.

In [ ]:
preproc.pos_tag()

We can now see a new column `pos` with the found POS tag for each token:

In [ ]:
preproc.tokens_datatable

### Aside: TMPreproc as "state machine"

Before continuing, we should clarify that a TMPreproc instance is a "state machine", i.e. its contents (the documents) and behavior can change when you call a method. An example:


```python
corpus = {
    "doc1": "Hello world!",
    "doc2": "Another example"
}

preproc = TMPreproc(corpus)     # documents are directly tokenized
preproc.tokens

# Out:
# {
#   'doc1': ['Hello', 'world', '!'],
#   'doc2': ['Another', 'example']
# }

preproc.tokens_to_lowercase()   # this changes the documents
preproc.tokens

# Out:
# {
#   'doc1': ['hello', 'world', '!'],
#   'doc2': ['another', 'example']
# }
```

As you can see, the tokens "inside" `preproc` are changed *in place*. It's important to see that after calling the method `tokens_to_lowercase()`, the tokens in `preproc` were transformed and the original tokens from before calling this method are not available anymore. In Python, assigning a *mutable* object to a variable binds the same object only to a different name, it doesn't copy it. Since a `TMPreproc` object is a mutable object (you can change its state by calling its methods), when we simply assign such an object to a different variable (say `preproc_upper`) we essentially only have two names for the same object and by calling a method on one of these variable names, the values will be changed for *both* names.

#### Copying `TMPreproc` objects

What can we do about that? We need to *copy* the object which can be done with the [TMPreproc.copy()](api.rst#tmtoolkit.preprocess.TMPreproc.copy) method. By this, we create another variable `preproc_upper` that points to a separate `TMPreproc` object.

In [ ]:
preproc_upper = preproc.copy()

In [ ]:
# the IDs confirm that we have two different objects
id(preproc_upper), id(preproc)

In [ ]:
preproc_upper.transform_tokens(str.upper)

# the transformation now only applied to "preproc_upper"
preproc.vocabulary == preproc_upper.vocabulary

In [ ]:
# show a sample
preproc_upper.tokens['NewsArticles-1880'][:10]

In [ ]:
# the original "preproc" still holds the same data
preproc.tokens['NewsArticles-1880'][:10]

Note that this also uses up twice as much computer memory now. So you shouldn't create copies that often and also release unused memory by using `del`:

In [ ]:
# removing the objects again
preproc_upper.shutdown_workers()
del preproc_upper

### Lemmatization and term normalization

Before we start with token normalization, we will create a copy of the original `TMPreproc` object and its data, so that we can later use it for comparison:

In [ ]:
preproc_orig = preproc.copy()

Lemmatization brings a token, if it is a word, to its base form. The lemma is already found out during the tokenization process and is available in the `lemma` metadata column. However, when you want to further process the tokens on the base of the lemmata, you should use the [lemmatize()](api.rst#tmtoolkit.preprocess.TMPreproc.lemmatize) method. This method sets the lemmata as tokens and all further processing will happen using the lemmatized tokens:

In [ ]:
preproc.lemmatize()
preproc.tokens_datatable

As we can see, the `lemma` column was copied over to the `token` column.

<div class="alert alert-info">

Stemming
    
tmtoolkit doesn't support stemming directly, since lemmatization is generally accepted as a better approach to bring different word forms of one word to a common base form. However, you may install [NLTK](https://www.nltk.org/) and apply stemming by using the [transform_tokens()](api.rst#tmtoolkit.preprocess.TMPreproc.transform_tokens) method together with the [stem()](api.rst#tmtoolkit.preprocess.stem) function.
    
</div>

Depending on how you further want to analyze the data, it may be necessary to "clean" or "normalize" your tokens in different ways in order to remove noise from the corpus, such as punctuation tokens or numbers, upper/lowercase forms of the same word, etc. Note that this is usually not necessary when you work with more modern approaches such as word embeddings (word vectors).   

If you want to remove certain characters in *all* tokens in your documents, you can use [remove_chars_in_tokens()](api.rst#tmtoolkit.preprocess.TMPreproc.remove_chars_in_tokens) and pass it a sequence of characters to remove. There is also a shortcut [remove_special_chars_in_tokens()](api.rst#tmtoolkit.preprocess.TMPreproc.remove_special_chars_in_tokens) which will remove all "special characters" (all characters in [string.punction](https://docs.python.org/3/library/string.html#string.punctuation) by default).

In [ ]:
preproc.remove_chars_in_tokens(['-'])  # remove only "-"
preproc.print_summary()

In [ ]:
# remove all punctuation
preproc.remove_special_chars_in_tokens()
preproc.print_summary()   # the "?" also vanishes

A common (but harsh) practice is to transform all tokens to lowercase forms, which can be done with [tokens_to_lowercase()](api.rst#tmtoolkit.preprocess.TMPreproc.tokens_to_lowercase):

In [ ]:
preproc.tokens_to_lowercase()
preproc.print_summary()

The method [clean_tokens()](api.rst#tmtoolkit.preprocess.TMPreproc.clean_tokens) finally applies several steps that remove tokens that meet certain criteria. This includes removing:

- punctuation tokens
- stopwords (very common words for the given language)
- empty tokens (i.e. `''`)
- tokens that are longer or shorter than a certain number of characters
- numbers  

Note that this is a language-dependent method, because the default stopword list is determined per language. This method has many parameters to tweak, so it's recommended to check out the documentation.

In [ ]:
# remove punct., stopwords, empty tokens (this is the default)
# plus tokens shorter than 2 characters and numeric tokens like "2019"
preproc.clean_tokens(remove_numbers=True, remove_shorter_than=2)
preproc.print_summary()

Due to the removal of several tokens in the previous step, the document lengths for the processed corpus are much smaller than for the original corpus:

In [ ]:
preproc.doc_lengths, preproc_orig.doc_lengths

We can also observe that the vocabulary got smaller after the processing steps, which, for large corpora, is also important in terms of computation time and memory consumption for later analyses:

In [ ]:
len(preproc.vocabulary), len(preproc_orig.vocabulary)

You can also apply custom token transform functions by using [transform_tokens()](api.rst#tmtoolkit.preprocess.TMpreproc.transform_tokens) and passing it a function that should be applied to each token in each document (hence it must accept one string argument).

First let's define such a function. Here we create a simple function that should return a token's "shape" in terms of the case of its characters:

In [ ]:
def token_shape(t):
    return ''.join(['X' if str.isupper(c) else 'x' for c in t])

token_shape('EU'), token_shape('CamelCase'), token_shape('lower')

We can now apply this function to our documents (we will use the original documents here, because they were not transformed to lower case):

In [ ]:
preproc.shutdown_workers()      # swap instances for later
preproc = preproc_orig.copy()

preproc_orig.transform_tokens(token_shape)   # apply function
preproc_orig.print_summary()

# remove instance
preproc_orig.shutdown_workers(); del preproc_orig

#### Expanding compound words and joining tokens

Compound words like "US-Student" or "non-recyclable" can be expanded to separate tokens like "US", "Student" and "non", "recyclable" using [expand_compound_tokens()](api.rst#tmtoolkit.preprocess.TMPreproc.expand_compound_tokens). However, depending on the language model, most of these compounds will already be separated on initial tokenization.

In [ ]:
orig_vocab = preproc.vocabulary
preproc.expand_compound_tokens()

# create set difference to show vocabulary tokens
# that were expanded
set(orig_vocab) - set(preproc.vocabulary)

It's also possible to join together certain *subsequent* occurrences of tokens or token patterns. This means you can for example transform all of the subsequent tokens "White" and "House" to single tokens "White_House". In case you don't use n-grams (described in a separate section), this is very helpful when you want to capture a named entity that is made up by several tokens, such as persons, institutions or concepts like "Climate Change", as a single token. The method to use for this is [glue_tokens()](api.rst#tmtoolkit.preprocess.TMPreproc.glue_tokens). It accepts the following parameters:

- a `patterns` sequence of length *N* that is used to match the subsequent *N* tokens;
- a `glue` string that is used to join the matched subsequent tokens (by default: `"_"`).

Along with that, you can adjust the token matching with the [common token matching parameters](#Common-parameters-for-pattern-matching-functions) described below.

Let's "glue" all subsequent occurrences of "White" and "House". The `glue_tokens()` method will return a set of glued tokens that matched the provided pattern:

In [ ]:
preproc_orig = preproc.copy()  # make a copy of full orig. data for later use
preproc.glue_tokens(['White', 'House'])

In [ ]:
preproc.tokens['NewsArticles-1880'][:20]

In [ ]:
preproc.shutdown_workers()

### Keywords-in-context (KWIC) and general filtering methods

*Keywords-in-context (KWIC)* allow you to quickly investigate certain keywords and their neighborhood of tokens, i.e. the tokens that appear right before and after this keyword.

`TMPreproc` provides three methods for this purpose:

- [get_kwic()](api.rst#tmtoolkit.preprocess.TMPreproc.get_kwic) is the base method accepting a search pattern and several options that control how the search pattern is matched (more on that below); use this function when you want to further process the output of a KWIC search;
- [get_kwic_table()](api.rst#tmtoolkit.preprocess.TMPreproc.get_kwic_table) is the more "user friendly" version of the above method as it produces a datatable with the highlighted keyword by default
- [filter_tokens_with_kwic()](api.rst#tmtoolkit.preprocess.TMPreproc.filter_tokens_with_kwic) works similar to the above functions but applies the result by filtering the documents again; it is explained in the [section on filtering](#Filtering-tokens-and-documents)

Let's see the KWIC methods in action:

In [ ]:
preproc = preproc_orig.copy()  # use orig. full data
preproc.get_kwic('house', ignore_case=True)

The method returns a dictionary that maps document labels to the KWIC results. Each document contains a list of "contexts", i.e. a list of tokens that surround a keyword, here `"house"`. This keyword stands in the middle and is surrounded by its "context tokens", which by default means two tokens to the left and two tokens to the right (which may be less when the keyword is near the start or the end of a document). 

We can see that `NewsArticles-1880` contains four contexts, `NewsArticles-99` one context and `NewsArticles-3350` none.

With `get_kwic_table()`, we get back a datatable which provides a better formatting for quick investigation. See how the matched tokens are highlighted as `*house*` and empty results are removed:

In [ ]:
preproc.get_kwic_table('house', ignore_case=True)

An important parameter is `context_size`. It determines the number of tokens to display left and right to the found keyword. You can either pass a single integer for a symmetric context or a tuple with integers `(<left>, <right>)`:

In [ ]:
preproc.get_kwic_table('house', ignore_case=True, context_size=4)

In [ ]:
preproc.get_kwic_table('house', ignore_case=True, context_size=(1, 4))

The KWIC functions become really powerful when using the pattern matching options. So far, we were looking for *exact* (but case insensitive) matches between the corpus tokens and our keyword `"house"`. However, it is also possible to match patterns like `"new*"` (matches any word starting with "new") or `"agenc(y|ies)"` (a regular expression matching "agency" and "agencies"). The next section gives an introduction on the different options for pattern matching.

#### Common parameters for pattern matching functions

Several functions and methods in tmtoolkit support pattern matching, including the already mentioned KWIC functions but also functions for filtering tokens or documents as you will see later. They all share similar function signatures, i.e. similar parameters:

- `search_token` or `search_tokens`: allows to specify one or more patterns as strings
- `match_type`: sets the matching type and can be one of the following options:
  - `'exact'` (default): exact string matching (optionally ignoring character case), i.e. no pattern matching
  - `'regex'` uses [regular expression](https://docs.python.org/3/library/re.html) matching
  - `'glob'` uses "glob patterns" like `"politic*"` which matches for example "politic", "politics" or "politician" (see [globre package](https://pypi.org/project/globre/))
- `ignore_case`: ignore character case (applies to all three match types)
- `glob_method`: if `match_type` is 'glob', use this glob method. Must be `'match'` or `'search'` (similar behavior as Python's [re.match](https://docs.python.org/3/library/re.html#re.match) or [re.search](https://docs.python.org/3/library/re.html#re.search))
- `inverse`: inverse the match results, i.e. if matching for "hello", return all results that do *not* match "hello"

Let's try out some of these options with `get_kwic_table()`:

In [ ]:
# using a regular expression, ignoring case
preproc.get_kwic_table(r'agenc(y|ies)', match_type='regex', ignore_case=True)

In [ ]:
# using a glob, ignoring case
preproc.get_kwic_table('pol*', match_type='glob', ignore_case=True)

In [ ]:
# using a glob, ignoring case
preproc.get_kwic_table('*sol*', match_type='glob', ignore_case=True)

In [ ]:
# using a regex that matches all tokens with at least one vowel and
# inverting these matches, i.e. all tokens *without* any vowels
preproc.get_kwic_table(r'[AEIOUaeiou]', match_type='regex', inverse=True)

#### Filtering tokens and documents

We can use the pattern matching parameters in numerous filtering methods. The heart of many of these methods is [token_match()](api.rst#tmtoolkit.preprocess.token_match). Given a search pattern, a list of tokens and optionally some pattern matching parameters, it returns a binary NumPy array of the same length as the input tokens. Each occurrence of `True` in this binary array signals a match.

In [ ]:
from tmtoolkit.preprocess import token_match

# first 10 tokens of document "NewsArticles-1880"
doc_snippet = preproc.tokens['NewsArticles-1880'][:10]
# get all tokens that match "to*"
matches = token_match('to*', doc_snippet, match_type='glob')

# iterate through tokens and matches, show pair-wise results
for tok, match in zip(doc_snippet, matches):
    print(tok, ':', match)

The `token_match()` function is a rather low-level function that you may use for pattern matching against any list/array of strings, e.g. a list of tokens, file names, etc.

The following methods cover common use-cases for filtering during text preprocessing. Many of these methods start either with `filter_...()` or `remove_...()` and these pairs of filter and remove functions are complements. A filter method will always *retain* the matched elements whereas a remove method will always *drop* the matched elements. We can observe that with the first pair of method, [filter_tokens()](api.rst#tmtoolkit.preprocess.TMPreproc.filter_tokens) and [remove_tokens()](api.rst#tmtoolkit.preprocess.TMPreproc.remove_tokens):

<div class="alert alert-info">

So much `.copy()`
    
Note that the following code snippets make lot of use of the `copy()` and `shutdown_workers()` methods. This is because we want to show how the different methods work with the *same original data* (remember that a `TMPreproc` instance behaves like a state machine) and also want to "clean up" the temporary instances. Under normal circumstances, you wouldn't use these two methods so excessively.
    
</div>

In [ ]:
# retain only the tokens that match the pattern in each document
preproc.filter_tokens('*house*', match_type='glob', ignore_case=True)\
    .print_summary().shutdown_workers()

In [ ]:
preproc = preproc_orig.copy()  # make a copy from full data

preproc.remove_tokens('*house*', match_type='glob', ignore_case=True)\
    .print_summary().shutdown_workers()

The pair [filter_documents()](api.rst#tmtoolkit.preprocess.TMPreproc.filter_documents) and [remove_documents()](api.rst#tmtoolkit.preprocess.TMPreproc.remove_documents) works similarily, but filters or drops whole documents regarding the supplied match criteria. Both accept the standard pattern matching parameters but also a parameter `matches_threshold` with default value `1`. When this number of matching tokens is hit, the document will be part of the result set (`filter_documents()`) or removed from the result set (`remove_documents()`). By this, we can for example retain only those documents that contain certain token patterns.

Let's try these methods out in practice:

In [ ]:
preproc = preproc_orig.copy()  # make a copy from full data

preproc.filter_documents('*house*', match_type='glob', ignore_case=True)\
    .print_summary().shutdown_workers()

We can see that two out of three documents contained the pattern `'*house*'` and hence were retained.

We can also adjust `matches_threshold` to set the minimum number of token matches for filtering:

In [ ]:
preproc = preproc_orig.copy()  # make a copy from full data

preproc.filter_documents('*house*', match_type='glob', ignore_case=True,
                         matches_threshold=4)\
    .print_summary().shutdown_workers()

In [ ]:
preproc = preproc_orig.copy()  # make a copy from full data

preproc.remove_documents('*house*', match_type='glob', ignore_case=True)\
    .print_summary().shutdown_workers()

When we use `remove_documents()` we get only the documents that did *not* contain the specified pattern.

Another useful pair of methods is [filter_documents_by_name()](api.rst#tmtoolkit.preprocess.TMPreproc.filter_documents_by_name) and [remove_documents_by_name()](api.rst#tmtoolkit.preprocess.TMPreproc.remove_documents_by_name). Both methods again accept the same pattern matching parameters but they only apply them to the document names, i.e. document *labels*:

In [ ]:
preproc = preproc_orig.copy()  # make a copy from full data

preproc.filter_documents_by_name(r'-\d{4}$', match_type='regex')\
    .print_summary().shutdown_workers()

In the above example we wanted to retain only the documents whose document labels ended with exactly 4 digits, like "...-1234". Hence, we only get "NewsArticles-1880" and "NewsArticles-3350" but not "NewsArticles-99". Again, `remove_documents_by_name()` will do the exact opposite.

You may also use [Keywords-in-context (KWIC)](#Keywords-in-context-(KWIC)) to filter your tokens in the neighborhood around certain keyword pattern(s). The method for that is called [filter_tokens_with_kwic()](api.rst#tmtoolkit.preprocess.TMPreproc.filter_tokens_with_kwic) and works very similar to [get_kwic()](api.rst#tmtoolkit.preprocess.get_kwic) but filters the documents in the `TMPreproc` instance with which you can continue working as usual. Here, we filter the tokens in each document to get the tokens directly in front and after the glob pattern `'*house*'` (`context_size=1`):

In [ ]:
preproc = preproc_orig.copy()  # make a copy from full data

preproc.filter_tokens_with_kwic('*house*', context_size=1,
                                match_type='glob', ignore_case=True)\
    .tokens_datatable

When you annotated your documents' tokens with Part-of-Speech (POS) tags, you can also filter them using [filter_for_pos()](api.rst#tmtoolkit.preprocess.TMPreproc.filter_for_pos):

In [ ]:
preproc = preproc_orig.copy()  # make a copy from full data

# apply POS tagging and retain only nouns
preproc.pos_tag().filter_for_pos('N').tokens_datatable

In [ ]:
preproc.shutdown_workers()

In this example we filtered for tokens that were identified as nouns by passing the *simplified POS tag* `'N'` (for more on simplified tags, see the method documentation). We can also filter for more than one tag, e.g. nouns or verbs by passing a list of required POS tags.

`filter_for_pos()` has no `remove_...()` counterpart, but you can set the `inverse` parameter to `True` to achieve the same effect.

Finally there are two methods for removing tokens based on their [document frequency](#Vocabulary-and-document-frequencies): [remove_common_tokens()](api.rst#tmtoolkit.preprocess.TMPreproc.remove_common_tokens) and [remove_uncommon_tokens()](api.rst#tmtoolkit.preprocess.TMPreproc.remove_uncommon_tokens). The former removes all tokens that have a document frequency greater or equal a certain threshold defined by parameter `df_threshold`. The latter does the same for all tokens that have a document frequency lower or equal `df_threshold`. This parameter can either be a relative frequency (default) or absolute count (by setting parameter `absolute=True`).

Before applying the method, let's have a look at the number of tokens per document again, to later see how many we will remove. We will also store the vocabulary in `orig_vocab` for later comparison:

In [ ]:
preproc = preproc_orig.copy()  # make a copy from full data
orig_vocab = preproc.vocabulary
preproc.doc_lengths

In [ ]:
preproc.remove_common_tokens(df_threshold=0.9).doc_lengths

By removing all tokens with a document frequency threshold of 0.9, we removed quite a number of tokens in each document. Let's investigate the vocabulary in order to see which tokens were removed:

In [ ]:
# set difference gives removed vocabulary tokens
set(orig_vocab) - set(preproc.vocabulary)

In [ ]:
preproc.shutdown_workers()

`remove_uncommon_tokens()` works similarily. This time, let's use an absolute number as threshold:

In [ ]:
preproc = preproc_orig.copy()  # make a copy from full data

preproc.remove_uncommon_tokens(df_threshold=1, absolute=True)\
    .shutdown_workers()
# set difference gives removed vocabulary tokens
# this time, show only the first 10 tokens that were removed
sorted(set(orig_vocab) - set(preproc.vocabulary))[:10]

The above means that we remove all tokens that appear only in exactly one document.

### Working with token metadata

`TMPreproc` allows to attach arbitrary metadata to each token in each document. This kind of "annotations" for tokens is very useful. For example, you may add metadata that records a token's length or whether it is all uppercase letters and later use that for filtering or in further analyses. One method to add such metadata is [add_metadata_per_doc()](api.rst#tmtoolkit.preprocess.TMPreproc.add_metadata_per_doc). This method requires to pass a dict that maps document labels to the respective token metadata list. The list's length must match the number of tokens in the respective document. At first we need to create such a metadata dict. Let's do that for the tokens' length first:

In [ ]:
preproc = preproc_orig.copy()  # make a copy from full data

meta_tok_lengths = {doc_label: list(map(len, doc_tokens))
                    for doc_label, doc_tokens in preproc.tokens.items()}

# show first 5 tokens and their string length for a sample document
list(zip(preproc.tokens['NewsArticles-1880'][:10],
         meta_tok_lengths['NewsArticles-1880'][:10]))

We can now add these metadata via [add_metadata_per_doc()](api.rst#tmtoolkit.preprocess.TMPreproc.add_metadata_per_doc). We pass a label, the metadata key, and the previously generated metadata:

In [ ]:
preproc.add_metadata_per_doc('length', meta_tok_lengths)
del meta_tok_lengths  # we don't need that object anymore

The property `.tokens_datatable` now shows an additional column with `meta_length` (the metadata key in always prefixed with `meta_`):

In [ ]:
preproc.tokens_datatable

Let's add a boolean indicator for whether the given token is all uppercase:

In [ ]:
meta_tok_upper = {doc_label: list(map(str.isupper, doc_tokens))
                  for doc_label, doc_tokens in preproc.tokens.items()}

preproc.add_metadata_per_doc('upper', meta_tok_upper)
del meta_tok_upper

preproc.tokens_datatable

You may use these newly added columns now for example for filtering the datatable:

In [ ]:
import datatable as dt

preproc.tokens_datatable[dt.f.meta_upper == 1,:]

To see which metadata keys were already created, you can use [get_available_metadata_keys()](api.rst#tmtoolkit.preprocess.TMPreproc.get_available_metadata_keys):

In [ ]:
preproc.get_available_metadata_keys()

Token metadata can be removed with [remove_metadata()](api.rst#tmtoolkit.preprocess.TMPreproc.remove_metadata):

In [ ]:
preproc.remove_metadata('upper')
preproc.get_available_metadata_keys()

In [ ]:
preproc.tokens_datatable

We can tell [filter_tokens()](api.rst#tmtoolkit.preprocess.TMPreproc.filter_tokens) and similar methods to use metadata instead of the tokens for matching. For example, we can use the metadata `meta_length`, which we created before, to filter for tokens of a certain length:

In [ ]:
preproc_meta_example = preproc.copy()
preproc_meta_example.filter_tokens(3, by_meta='length')\
    .tokens_datatable

In [ ]:
preproc_meta_example.shutdown_workers(); del preproc_meta_example

Note that all matching options then apply to the metadata column, in this case to the `meta_length` column which contains integers. Since `filter_tokens()` by default employs exact matching, we get all tokens where `meta_length` equals the first argument, `3`. If we used regular expression or glob matching instead, this method would fail because you can only use that for string data.

If you want to use more complex filter queries, you should create a "filter mask" and pass it to [filter_tokens_by_mask()](api.rst#tmtoolkit.preprocess.TMPreproc.filter_tokens_by_mask). A filter mask is a dictionary that maps a document label to a sequence of booleans. For all occurrences of `True`, the respective token in the document will be retained, all others will be removed. Let's try that out with a small sample:

In [ ]:
preproc.pos_tag().tokens_datatable

We now generate the filter mask, which means for each document we create a boolean list or array that for each token in that document indicates whether that token should be kept or removed.

We will iterate through the [tokens_with_metadata](api.rst#tmtoolkit.preprocess.TMPreproc.tokens_with_metadata) property which is a dict that for each document contains a datatable with its tokens and metadata. Let's have a look at the first document's datatable:

In [ ]:
next(iter(preproc.tokens_with_metadata.values()))

Now we can create the filter mask:

In [ ]:
import numpy as np

filter_mask = {}
for doc_label, doc_data in preproc.tokens_with_metadata.items():
    # extract the columns "meta_length" and "pos"
    # and convert them to NumPy arrays
    doc_data_subset = doc_data[:, [dt.f.meta_length, dt.f.pos]]
    tok_lengths, tok_pos = map(np.array, doc_data_subset.to_list())
    
    # create a boolean array for nouns with token length less or equal 5
    filter_mask[doc_label] = (tok_lengths <= 5) & np.isin(tok_pos, ['NOUN', 'PROPN'])

# it's not necessary to add the filter mask as metadata
# but it's a good way to check the mask
preproc.add_metadata_per_doc('small_nouns', filter_mask)
preproc.tokens_datatable

Finally, we can pass the mask dict to [filter_tokens_by_mask()](api.rst#tmtoolkit.preprocess.TMPreproc.filter_tokens_by_mask):

In [ ]:
preproc.filter_tokens_by_mask(filter_mask)
preproc.tokens_datatable

### Generating n-grams

So far, we worked with *unigrams*, i.e. each document consisted of a sequence of discrete tokens. We can also generate *n-grams* from our corpus where each document consists of a sequence of *n* subsequent tokens. An example would be:

Document: "This is a simple example."

**n=1 (unigrams):**

    ['This', 'is', 'a', 'simple', 'example', '.']

**n=2 (bigrams):**

    ['This is', 'is a', 'a simple', 'simple example', 'example .']

**n=3 (trigrams):**

    ['This is a', 'is a simple', 'a simple example', 'simple example .']

The method [generate_ngrams()](api.rst#tmtoolkit.preprocess.TMPreproc.generate_ngrams) allows us to generate n-grams from tokenized documents. We can then get the results with the `ngrams` property:

In [ ]:
preproc = preproc_orig.copy()  # make a copy from full data

preproc.generate_ngrams(2)  # generate bigrams
preproc.ngrams['NewsArticles-1880'][:10]  # show first 10 bigrams of this document

In [ ]:
preproc.shutdown_workers(); del preproc

### Generating a sparse document-term matrix (DTM)

If you're working with a bag-of-words representation of your data, you usually convert the preprocessed documents to a document-term matrix (DTM), which represents of the number of occurrences of each term (i.e. vocabulary token) in each document. This is a *N* rows by *M* columns matrix, where *N* is the number of documents and *M* is the vocabulary size (i.e. the number of unique tokens in the corpus).

Not all tokens from the vocabulary occur in all documents. In fact, many tokens will occur only in a small subset of the documents if you're dealing with a "real world" dataset. This means that most entries in such a DTM will be zero. Almost all functions in tmtoolkit therefore generate and work with *sparse* matrices, where only non-zero values are stored in computer memory.

For this example, we'll generate a DTM from the `preproc_orig` instance. First, let's check the number of documents and the vocabulary size:

In [ ]:
preproc_orig.n_docs, preproc_orig.vocabulary_size

We can use the [dtm](api.rst#tmtoolkit.preprocess.TMPreproc.dtm) property to generate a sparse DTM from the current instance:

In [ ]:
preproc_orig.dtm

We can see that a sparse matrix with 3 rows (which corresponds with the number of documents) and 683 columns was generated (which corresponds with the vocabulary size). 816 elements in this matrix are non-zero.

We can convert this matrix to a non-sparse, i.e. *dense*, representation and see parts of its elements:

In [ ]:
preproc_orig.dtm.todense()

However, note that you should only convert a sparse matrix to a dense representation when you're either dealing with a small amount of data (which is what we're doing in this example), or use only a part of the full matrix. Converting a sparse matrix to a dense representation can otherwise easily exceed the available computer memory.

There exist different "formats" for sparse matrices, which have different advantages and disadvantes (see for example the [SciPy "sparse" module documentation](https://docs.scipy.org/doc/scipy/reference/sparse.html#usage-information). **Not all formats support all operations that you can usually apply to an ordinary, dense matrix.** By default, the generated DTM is in *Compressed Sparse Row (CSR)* format. This format allows indexing and is especially optimized for fast row access. You may convert it to any other sparse matrix format; see the mentioned SciPy documentation for this.

The rows of the DTM are aligned to the sequence of the document labels and its columns are aligned to the vocabulary. For example, let's find the frequency of the term "House" in the document "NewsArticles-1880". To do this, we find out the indices into the matrix:

In [ ]:
preproc_orig.doc_labels.index('NewsArticles-1880')

In [ ]:
preproc_orig.vocabulary.index('House')

This means the frequency of the term "House" in the document "NewsArticles-1880" is located in row 0 and column 4 of the DTM:

In [ ]:
preproc_orig.dtm[0, 67]

See also the following example of finding out the index for "administration" and then getting an array that represents the number of occurrences of this token across all three documents:

In [ ]:
vocab_admin_ix = preproc_orig.vocabulary.index('administration')
preproc_orig.dtm[:, vocab_admin_ix].todense()

Apart from the [dtm](api.rst#tmtoolkit.preprocess.TMPreproc.dtm) property, there's also the [get_dtm](api.rst#tmtoolkit.preprocess.TMPreproc.get_dtm) method which allows to also return the result as datatable or pandas DataFrame. Note that these representations are not sparse and hence can consume much memory.

In [ ]:
preproc_orig.get_dtm(as_datatable=True)

### Serialization: Saving and loading `TMPreproc` objects

The current state of a `TMPreproc` object can also be stored to a file on disk so that you (or someone else who has tmtoolkit installed) can later restore it using that file. The methods for that are [save_state()](api.rst#tmtoolkit.preprocess.TMPreproc.save_state) and [load_state()](api.rst#tmtoolkit.preprocess.TMPreproc.load_state) / [from_state()](api.rst#tmtoolkit.preprocess.TMPreproc.from_state).

Let's store the current state of the `preproc_orig` instance:

In [ ]:
preproc_orig.print_summary()
preproc_orig.save_state('data/preproc_state.pickle')

Let's change the object by retaining only documents that contain the token "house" (see the reduced number of documents):

In [ ]:
preproc_orig.filter_documents('*house*', match_type='glob', ignore_case=True)
preproc_orig.print_summary()

We can restore the saved data using [from_state()](api.rst#tmtoolkit.preprocess.TMPreproc.from_state):

In [ ]:
preproc_restored = TMPreproc.from_state('data/preproc_state.pickle')
preproc_restored.print_summary()

You can see that the full dataset with three documents was restored.

This is very useful especially when you have a large amount of data and run time consuming operations, e.g. POS tagging. When you're finished running these operations, you can easily store the current state to disk and later retrieve it without the need to re-run these operations.

## Functional API

The `TMPreproc` class provides a convenient object-oriented interface for parallel text processing and analysis. There is also a *functional API* provided in the [tmtoolkit.preprocess](api.rst#tmtoolkit-preprocess) module. Most of these functions accept a list of spaCy documents along with additional parameters. You may use these functions for quick prototyping, but it is generally much more convenient to use `TMPreproc`. The functional API does not provide parallel processing.

To initialize the functional API for a certain language, you need to start with [init_for_language()](api.rst#tmtoolkit.preprocess.init_for_language) and may then tokenize your raw text documents via [tokenize()](api.rst#tmtoolkit.preprocess.tokenize), which will generate a list of spaCy documents. Most other functions in this API accept such a list of list of spaCy documents as input.

```
init_for_language('en')
docs = tokenize(['Hello this is a test.', 'And here comes another one.'])
```

---

The final result after applying preprocessing steps and hence transforming the text data, is often a document-term matrix (DTM). The [bow module](api.rst#tmtoolkit-bow) contains several functions to work with DTMs, e.g. apply transformations such as *tf-idf* or compute some important summary statistics. The [next chapter](bow.ipynb) will introduce some of these functions.